In [2]:
# ZJZ twitter sentiment analysis

In [3]:
# import
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [4]:
accessToken='xxxx'
accessTokenSecret='xxxx'
consumerKey='xxxx'
consumerSecret='xxxx'
auth = tweepy.OAuthHandler(consumerKey,consumerSecret)
auth.set_access_token(accessToken,accessTokenSecret)
api=tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# 提取100条tweets 
posts = api.user_timeline(screen_name='BillGates',count=100,lang="en",tweet_mode="extended")
print("show the n recent tweets:\n")
i = 0
for tweet in posts[0:5]:
  i+=1
  print(str(i)+')'+tweet.full_text+'\n')

In [ ]:
df = pd.DataFrame([tweet.full_text for tweet in posts],columns=['Tweets'])
df.head()

In [ ]:
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+','',text)
  text = re.sub(r'#','',text)
  text = re.sub(r'RT[\s]+','',text)
  text = re.sub(r'https:\/\/S+','',text)
  return(text)

df['Tweets']=df['Tweets'].apply(cleanTxt)
df

In [ ]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

df['Subjectivity']=df['Tweets'].apply(getSubjectivity)
df['Polarity']=df['Tweets'].apply(getPolarity)
df

In [ ]:
# word cloud
allWords = ' '.join([twts for twts in df['Tweets']])
wordCloud = WordCloud(width = 500,height = 300, random_state=80, max_font_size=100).generate(allWords)
plt.imshow(wordCloud,interpolation='bilinear')
plt.axis('on')
plt.show()

In [ ]:
#NLP part
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score==0:
    return 'Middle'
  else:
    return 'Positive'
df['Analysis']=df['Polarity'].apply(getAnalysis)
df

In [ ]:
j=1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0,sortedDF.shape[0]):
  if(sortedDF['Analysis'][i] == 'Positive'):
    print(str(j)+')'+sortedDF['Tweets'][i])
    print()
    j=j+1

In [ ]:
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
  plt.scatter(df['Polarity'][i],df['Subjectivity'][i],color='Blue')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show